### w2v50をテキストにする

In [1]:
cd ../data/external/

/home/saho/project/wovle/data/external


In [4]:
ls

1gm-0000   csj.pdp.htkdic       filelist    w2v200.txt
CJLC-0.1/  dictation-kit-v4.4/  index.html  w2v50/


In [5]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('w2v50/word2vec.gensim.model')
len(model.wv.vocab)

335476

In [6]:
with open('w2v50.tsv','w') as fh:
    for (k,v) in monitor(model.wv.vocab.items()):
        fh.write(k+"\t"+"\t".join([str(e) for e in model.wv[k]])+"\n")

100%|██████████| 335476/335476 [00:38<00:00, 8754.04it/s]


In [9]:
rm -r w2v50/

In [10]:
ls

1gm-0000   csj.pdp.htkdic       filelist    w2v200.txt
CJLC-0.1/  dictation-kit-v4.4/  index.html  w2v50.tsv


### 辞書（csj.pdp.htkdic）に無いものは削除

In [12]:
!nkf -w ./csj.pdp.htkdic > ./csj.pdp.htkdic.utf8

In [13]:
!head ./csj.pdp.htkdic.utf8

In [15]:
words = set()

with open('csj.pdp.htkdic.utf8') as hh:
    for line in monitor(hh):
        words.add(line.split('\t')[0].split('+')[0])

with open('w2v50.tsv') as v50h:
    with open('w2v200.txt') as v200h:
        with open('w2v50_fitToDic.tsv','w') as w50h:
            with open('w2v200_fitToDic.tsv','w') as w200h:
                for line in monitor(v50h):
                    w = line.split('\t')[0]
                    if w in words:
                        w50h.write(w+'\t'+'\t'.join(line.split('\t')[1:]))
                for line in monitor(v200h):
                    w = line.split(' ')[0].replace('[','').replace(']','') # 括弧は無いものとして比較
                    if w in words:
                        w200h.write(w+'\t'+'\t'.join(line.split(' ')[1:]))


0it [00:00, ?it/s]
31805it [00:00, 315776.63it/s]
38598it [00:00, 323445.17it/s]
0it [00:00, ?it/s]
4686it [00:00, 46453.69it/s]
8381it [00:00, 41621.04it/s]
13223it [00:00, 43669.20it/s]
20153it [00:00, 50270.04it/s]
27160it [00:00, 54232.77it/s]
33504it [00:00, 55768.65it/s]
39741it [00:00, 56679.34it/s]
46522it [00:00, 58077.45it/s]
55364it [00:00, 61432.10it/s]
63760it [00:01, 63686.60it/s]
72640it [00:01, 65979.70it/s]
80493it [00:01, 67009.62it/s]
88534it [00:01, 68048.79it/s]
97021it [00:01, 69234.76it/s]
105284it [00:01, 70113.05it/s]
115121it [00:01, 71902.74it/s]
125586it [00:01, 73821.36it/s]
134769it [00:01, 74690.39it/s]
144768it [00:01, 76020.28it/s]
155104it [00:02, 77332.01it/s]
164760it [00:02, 77990.85it/s]
174198it [00:02, 78663.61it/s]
185883it [00:02, 80297.46it/s]
195958it [00:02, 80918.92it/s]
206402it [00:02, 81856.50it/s]
216876it [00:02, 82728.39it/s]
228531it [00:02, 83960.98it/s]
240373it [00:02, 85188.94it/s]
251808it [00:02, 86177.59it/s]
263377it [00:03,

591360it [00:21, 27465.01it/s]
594667it [00:21, 27481.23it/s]
597909it [00:21, 27504.41it/s]
601913it [00:21, 27561.25it/s]
606233it [00:21, 27632.79it/s]
610291it [00:22, 27691.77it/s]
614349it [00:22, 27749.94it/s]
618561it [00:22, 27814.42it/s]
622591it [00:22, 27869.86it/s]
626593it [00:22, 27916.07it/s]
630569it [00:22, 27968.14it/s]
634509it [00:22, 28014.00it/s]
638404it [00:22, 28049.43it/s]
642191it [00:22, 28086.04it/s]
646554it [00:22, 28153.86it/s]
650474it [00:23, 28200.09it/s]
654379it [00:23, 28217.19it/s]
658143it [00:23, 28257.67it/s]
662228it [00:23, 28308.44it/s]
666044it [00:23, 28341.26it/s]
669869it [00:23, 28383.09it/s]
673916it [00:23, 28433.92it/s]
677837it [00:23, 28479.19it/s]
681719it [00:23, 28522.30it/s]
685587it [00:24, 28559.75it/s]
689405it [00:24, 28593.42it/s]
693168it [00:24, 28624.76it/s]
697255it [00:24, 28675.18it/s]
701076it [00:24, 28711.45it/s]
705145it [00:24, 28760.29it/s]
709028it [00:24, 28800.71it/s]
712906it [00:24, 28823.05it/s]
717011it

In [16]:
!wc -l ./w2v50_fitToDic.tsv

28023 ./w2v50_fitToDic.tsv


In [17]:
!wc -l ./w2v200_fitToDic.tsv

43929 ./w2v200_fitToDic.tsv


### w2v200の括弧重複の平均を取る

#### dataframeとして読み込み

In [76]:
w2v50 = pd.read_csv("w2v50_fitToDic.tsv",sep="\t",header=None)

In [77]:
w2v200 = pd.read_csv("w2v200_fitToDic.tsv",sep="\t",header=None)

In [78]:
w2v = [w2v50, w2v200]

In [79]:
for w in w2v: display(w.shape)

(28023, 51)

(43929, 201)

In [80]:
w2v[0].to_pickle('w2v50.pd')

In [81]:
w2v[1].to_pickle('w2v200.pd')

#### 重複除外

In [82]:
w = w2v[1]

In [83]:
def disp(*v):
    for e in v: display(e)

In [84]:
disp(
    w.ix[[0],:],
    w.shape)

/home/saho/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,の,-0.991739,0.433873,-1.634327,-2.637119,1.87341,-0.68232,1.400241,2.555166,-2.150506,...,-1.206327,0.912507,-0.449425,-0.685022,-0.067702,-0.439921,-1.823313,-0.582093,-0.289067,-0.953114


(43929, 201)

In [85]:
w.ix[w[0] == "トヨタ"]

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
7158,トヨタ,0.053538,-0.458620,2.269142,2.213828,1.045843,-0.385347,1.634038,2.483560,-1.511059,...,-0.565592,-1.320724,1.17779,-0.255538,1.393715,-0.182194,0.562751,-0.150641,0.373629,0.638203
18038,トヨタ,-0.306716,-1.427696,1.110670,1.346888,-0.350039,-0.451238,1.047367,1.017068,-1.072866,...,-0.431226,-0.514442,1.05925,-0.159481,0.126721,-0.407040,-0.150029,0.352745,0.327678,-0.481530


In [86]:
w = w.groupby([0]).mean()

In [87]:
w.head()

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0,,,,,,,,,,,,,,,,,,,,,
Β,-0.163973,0.013051,0.130042,-0.013135,-0.078542,0.067389,-0.027251,0.070931,0.024434,-0.089123,...,0.081253,-0.051911,0.031374,0.011905,-0.129501,0.059161,-0.033802,-0.107828,-0.159823,0.072723
Γ,-0.515290,0.447887,-0.290260,-0.313667,-0.217771,-0.583412,-0.237821,0.986688,0.573613,-0.021468,...,-0.319555,-0.453109,0.819951,-0.046766,-0.202213,0.104598,-0.665681,-0.421789,-0.654559,-0.389066
Δ,-0.844599,0.500375,-0.433498,-0.863186,-0.685131,-0.208649,0.063803,1.021694,0.299519,-0.326941,...,0.101914,-0.068191,0.600853,-0.186434,-0.277834,-0.085344,-1.251228,-1.181830,-1.252247,-0.060531
Θ,-0.146901,-0.095641,-0.023297,-0.016684,-0.133185,0.047843,0.085798,0.360042,0.040978,-0.093064,...,0.115077,-0.086670,0.071753,-0.126756,-0.154439,0.117072,-0.070839,-0.238478,-0.136663,-0.044030
Κ,-0.062942,0.024537,0.062008,-0.046116,0.023443,0.035645,0.009635,0.086242,0.075110,-0.041233,...,0.034198,-0.074562,0.040172,-0.019908,-0.092042,0.006126,-0.026464,-0.126889,-0.097553,-0.009375


In [88]:
w.shape

(29716, 200)

In [89]:
w.to_pickle('./w2v200.pd')

In [90]:
w = w2v[0]

In [91]:
w.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,ガリバー,0.091114,-0.294387,0.021036,-0.036468,0.101153,0.058209,0.147456,0.059062,0.146042,...,0.239939,-0.006300,0.099647,0.036027,-0.008679,0.039122,-0.049217,0.203507,0.180088,-0.169165
1,くん,0.060573,-0.231098,0.012265,-0.118655,-0.045415,-0.082472,0.005965,-0.031879,0.066911,...,0.133983,-0.022876,0.006493,0.064899,-0.067395,-0.020905,-0.154779,0.175292,0.191491,-0.281831
2,は,0.206208,0.238194,0.164277,-0.219658,0.163085,-0.229095,0.174790,0.049674,-0.146848,...,-0.105047,-0.001622,0.188861,-0.127376,0.117224,0.107232,0.254831,0.029983,0.024894,0.234945
3,で,-0.095309,0.110498,0.080374,-0.083534,0.010353,0.085707,-0.014444,0.092069,-0.066631,...,0.191288,-0.000440,0.043671,-0.194064,-0.043659,-0.054249,0.276505,0.072885,-0.139471,0.050859
4,放送,0.026639,0.180496,0.039786,-0.046034,0.125449,0.045694,0.132464,-0.216484,0.064253,...,-0.036977,-0.021256,-0.281305,0.142562,-0.270052,-0.148703,0.058203,0.050531,0.025959,0.074091


In [92]:
w = w.groupby([0]).mean()

In [95]:
w.shape

(28023, 50)

In [96]:
w.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
Β,-0.039774,-0.182654,-0.073650,-0.154080,-0.299761,0.039800,0.075907,0.151008,-0.040061,-0.092438,...,0.063181,0.152201,-0.012748,-0.189813,0.159146,-0.035278,-0.095846,-0.078241,0.011627,0.039851
Γ,-0.078112,-0.044997,-0.206049,0.121543,-0.158654,0.089009,-0.259112,0.045027,-0.001551,0.029611,...,0.099573,0.219084,0.041698,-0.191238,0.180090,0.019458,0.096816,0.014848,-0.216477,0.112108
Δ,-0.084763,0.060476,-0.208180,-0.032915,-0.070210,0.075549,-0.241700,0.069920,0.119192,-0.040702,...,0.037314,0.198016,-0.073684,-0.092430,0.135085,-0.068829,-0.092230,0.004051,-0.205020,0.138573
Θ,0.020196,-0.070559,-0.016031,-0.095259,-0.210722,0.008944,-0.102367,0.049041,0.136521,0.023940,...,0.191656,0.182939,0.138991,-0.162631,0.235224,0.048330,-0.076787,0.091762,0.008522,0.029727
Κ,-0.040871,-0.122597,-0.064304,-0.238372,-0.136569,0.085343,0.046668,0.087554,0.072072,-0.035176,...,0.101782,0.278786,0.114090,-0.252151,0.110471,0.204966,-0.032274,0.070054,-0.002045,0.014650


In [97]:
w.to_pickle('./w2v50.pd')

In [98]:
w2v = []


In [99]:
w2v.append(pd.read_pickle('w2v50.pd'))
w2v.append(pd.read_pickle('w2v200.pd'))

### 非自立語の削除

In [100]:
for (i,w) in enumerate(w2v):
    with open(str(i),"w") as fh:
        for e in w.index:
            fh.write(e+"\n")

In [101]:
!mecab 0 > 0.mecab
!mecab 1 > 1.mecab

In [102]:
!head 0.mecab

Β	記号,アルファベット,*,*,*,*,Β,ベータ,ベータ
EOS
Γ	記号,アルファベット,*,*,*,*,Γ,ガンマ,ガンマ
EOS
Δ	記号,アルファベット,*,*,*,*,Δ,デルタ,デルタ
EOS
Θ	記号,アルファベット,*,*,*,*,Θ,シータ,シータ
EOS
Κ	記号,アルファベット,*,*,*,*,Κ,カッパ,カッパ
EOS


In [103]:
!head 1.mecab

Β	記号,アルファベット,*,*,*,*,Β,ベータ,ベータ
EOS
Γ	記号,アルファベット,*,*,*,*,Γ,ガンマ,ガンマ
EOS
Δ	記号,アルファベット,*,*,*,*,Δ,デルタ,デルタ
EOS
Θ	記号,アルファベット,*,*,*,*,Θ,シータ,シータ
EOS
Κ	記号,アルファベット,*,*,*,*,Κ,カッパ,カッパ
EOS


In [104]:
 !cat 0.mecab |\
    grep -v "EOS" |\
    grep -v "助詞" |\
    grep -v "助動詞" |\
    grep -v "記号" |\
    grep -v "フィラー" |\
    grep -v "感動詞" |\
    grep -v "接頭詞" |\
    grep -v "接続詞" |\
    grep -v "連体詞" |\
    grep -v "副詞" |\
    grep -v "代名詞" |\
    grep -v "非自立" |\
    grep -v "この" \
    > 0.filted

In [105]:
 !cat 1.mecab |\
    grep -v "EOS" |\
    grep -v "助詞" |\
    grep -v "助動詞" |\
    grep -v "記号" |\
    grep -v "フィラー" |\
    grep -v "感動詞" |\
    grep -v "接頭詞" |\
    grep -v "接続詞" |\
    grep -v "連体詞" |\
    grep -v "副詞" |\
    grep -v "代名詞" |\
    grep -v "非自立" |\
    grep -v "この" \
    > 1.filted

In [106]:
!wc -l 0.filted

26931 0.filted


In [107]:
!wc -l 1.filted

28642 1.filted


In [108]:
w2v_word = [[],[]]
with open('0.filted') as fh:
    w2v_word[0] = [e.split('\t')[0] for e in fh]
with open('1.filted') as fh:
    w2v_word[1] = [e.split('\t')[0] for e in fh]
        

In [109]:
for w in w2v: display(w.shape)

(28023, 50)

(29716, 200)

In [110]:
w2v[0] = w2v[0][w2v[0].index.isin(w2v_word[0])]

In [111]:
w2v[1] = w2v[1][w2v[1].index.isin(w2v_word[1])]

In [112]:
for w in w2v: display(w.shape)

(26642, 50)

(27910, 200)

In [113]:
w2v[0].to_pickle('./w2v50.pd')
w2v[1].to_pickle('./w2v200.pd')